### Example performing two simple ensemble estimations: 1. statistics-maximum; 2. coherence-semblance 

#### Import required modules

In [ ]:
from os import listdir
from os.path import isfile, join
from operator import attrgetter
from time import time
import numpy as np
from obspy import UTCDateTime
from obspy.clients.fdsn import Client
from obspy.io.sac.core import _read_sac
from obspy.clients.fdsn.header import FDSNException
import matplotlib.pyplot as plt
import pandas as pd
import sys
import copy
import obspy
from obspy.core.stream import Stream
from obspy import UTCDateTime
from obspy import read
import time

from logging import raiseExceptions
import seisbench.models as sbm
import torch

sys.path.append('../../ELEP/src/')
from mbf_utils import make_LogFq, make_LinFq, rec_filter_coeff, create_obspy_trace
from mbf import MB_filter as MBF

#### load data from base predictions

# dir to load data
dirpath_csv = #os.path.join(dirpath, 'data', 'csv')
dirpath_h5 = #os.path.join(dirpath, 'data', 'h5')


#### perform ensemble-maximum and coherence-semblance estimations